In [1]:
import numpy as np

#import matplotlib.pyplot as plt
#from bokeh.io import push_notebook, show, output_notebook; 
#from bokeh.plotting import figure; 
#from bokeh.layouts import column
#output_notebook()

import USGShydro


to do:

USGShydro.py
- ~~ownload rainfall~~ 
- ~~download runoff~~
- merge data capability?
- convert data to numpy array

RainfallRunoff.py
- create time series dataset of lags, etc.
- drop observations with missing values
- run rainfall-runoff model
- plot predicted runoff vs. actual
- predict days above flood stage with models trained over different time periods

-universalize the procedure for other sites
-- create tool to query sites for dates and availability of needed series
-compare sites and see if model relates to hydrologic featues, basin topography, landcover, etc.

In [2]:
site='sites=02314495'
start='startDT=1951-01-01T00:00%2b0000'
start='startDT=2019-05-18T00:00%2b0000'#short interval for practice
end='endDT=2019-05-19T06:56-0400'
#data1=getUSGS.Get_data(site, start, end)
paramlist=['00065', '00045'] #must be entered as strings
modelfeatures=[]
try1=USGShydro.Hydrositedatamodel(site,start,end,paramlist,modelfeatures)

a
b


In [3]:
try1.get_data()

In [4]:
i=-1;j=-1;k=-1;m=-1
for child in try1.root:
        i+=1;print('i=',i);j=0
        print('parent:',child.tag)#,'#', child.attrib)
        for child2 in child:
            j+=1;print('i=',i,'j=', j);k=0
            print('child:',child2.tag)#,'#', child2.attrib)
            for child3 in child2:
                k+=1; print('i=',i,'j=', j,'k=',k)
                print('grandchild:',child3.tag)#,'#', child3.attrib)
                for child4 in child3:
                    k+=1; print('i=',i,'j=', j,'k=',k, 'm=',m)
                    print('ggrandchild:',child4.tag)#,'#', child3.attrib)
                    for child5 in child4:
                        print(child5.tag)
                
                

i= 0
parent: {http://www.opengis.net/gml/3.2}identifier
i= 1
parent: {http://www.opengis.net/gml/3.2}name
i= 2
parent: {http://www.opengis.net/waterml/2.0}metadata
i= 2 j= 1
child: {http://www.opengis.net/waterml/2.0}DocumentMetadata
i= 2 j= 1 k= 1
grandchild: {http://www.opengis.net/gml/3.2}metaDataProperty
i= 2 j= 1 k= 2
grandchild: {http://www.opengis.net/waterml/2.0}generationDate
i= 2 j= 1 k= 3
grandchild: {http://www.opengis.net/waterml/2.0}version
i= 3
parent: {http://www.opengis.net/waterml/2.0}observationMember
i= 3 j= 1
child: {http://www.opengis.net/om/2.0}OM_Observation
i= 3 j= 1 k= 1
grandchild: {http://www.opengis.net/om/2.0}phenomenonTime
i= 3 j= 1 k= 2 m= -1
ggrandchild: {http://www.opengis.net/gml/3.2}TimePeriod
{http://www.opengis.net/gml/3.2}beginPosition
{http://www.opengis.net/gml/3.2}endPosition
i= 3 j= 1 k= 3
grandchild: {http://www.opengis.net/om/2.0}resultTime
i= 3 j= 1 k= 4 m= -1
ggrandchild: {http://www.opengis.net/gml/3.2}TimeInstant
{http://www.opengis.net/

In [20]:
i=-1;j=-1;k=-1;m=-1
for child in try1.root:
        i+=1;print('i=',i);j=-1
        print('parent:',child.text)#,'#', child.attrib)
        for child2 in child:
            j+=1;print('i=',i,'j=', j);k=-1
            print(child2.text,'###',child2.attrib,'###', child2.tag)
            for child3 in child2:
                k+=1; print('i=',i,'j=', j,'k=',k);m=-1
                print('grandchild:',child3.text)#,'#', child3.attrib)
                for child4 in child3:
                    k+=1; print('i=',i,'j=', j,'k=',k, 'm=',m)
                    print('ggrandchild:',child4.text)#,'#', child3.attrib)
                    for child5 in child4:
                        print(child5.text)
                        for child6 in child5:
                            print(child6.text)

i= 0
parent: USGS.02314495
i= 1
parent: Timeseries collected at SUWANNEE RIVER ABOVE FARGO, GA
i= 2
parent: None
i= 2 j= 0
None ### {'{http://www.opengis.net/gml/3.2}id': 'doc.USGS.MP.USGS.02314495'} ### {http://www.opengis.net/waterml/2.0}DocumentMetadata
i= 2 j= 0 k= 0
grandchild: None
i= 2 j= 0 k= 1
grandchild: 2019-06-03T07:27:51.005-04:00
i= 2 j= 0 k= 2
grandchild: None
i= 3
parent: None
i= 3 j= 0
None ### {'{http://www.opengis.net/gml/3.2}id': 'obs.USGS.02314495.00045.228515.00000'} ### {http://www.opengis.net/om/2.0}OM_Observation
i= 3 j= 0 k= 0
grandchild: None
i= 3 j= 0 k= 1 m= -1
ggrandchild: None
2019-05-18T00:00:00-04:00
2019-05-19T10:45:00-04:00
i= 3 j= 0 k= 2
grandchild: None
i= 3 j= 0 k= 3 m= -1
ggrandchild: None
2019-05-19T10:45:00-04:00
i= 3 j= 0 k= 4
grandchild: None
i= 3 j= 0 k= 5 m= -1
ggrandchild: None
None
None
None
i= 3 j= 0 k= 6
grandchild: None
i= 3 j= 0 k= 7
grandchild: None
i= 3 j= 0 k= 8 m= -1
ggrandchild: None
None
None
None
None
i= 3 j= 0 k= 9
grandchild: 

In [6]:
'''print(try1.root[3][1][1][0].tag)
print(try1.root[3][0][1][0][0].text)
#print(try1.root[3][0][2][0][0].value)

print(try1.root[3][1][5])
print(try1.root[3][0][1][0][0].text)
print(try1.root[3][0][2][0][0].value)
'''

'print(try1.root[3][1][1][0].tag)\nprint(try1.root[3][0][1][0][0].text)\n#print(try1.root[3][0][2][0][0].value)\n\nprint(try1.root[3][1][5])\nprint(try1.root[3][0][1][0][0].text)\nprint(try1.root[3][0][2][0][0].value)\n'

In [22]:
print(try1.root[2].attrib)

{}


In [26]:
[(elem.tag,elem.attrib,elem.text) for elem in try1.root.iter()]

[('{http://www.opengis.net/waterml/2.0}Collection',
  {'{http://www.opengis.net/gml/3.2}id': 'C.USGS.02314495',
   '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.opengis.net/waterml/2.0 http://schemas.opengis.net/waterml/2.0/waterml2.xsd'},
  None),
 ('{http://www.opengis.net/gml/3.2}identifier',
  {'codeSpace': 'http://waterservices.usgs.gov/nwis/iv'},
  'USGS.02314495'),
 ('{http://www.opengis.net/gml/3.2}name',
  {'codeSpace': 'http://waterservices.usgs.gov/nwis/iv'},
  'Timeseries collected at SUWANNEE RIVER ABOVE FARGO, GA'),
 ('{http://www.opengis.net/waterml/2.0}metadata', {}, None),
 ('{http://www.opengis.net/waterml/2.0}DocumentMetadata',
  {'{http://www.opengis.net/gml/3.2}id': 'doc.USGS.MP.USGS.02314495'},
  None),
 ('{http://www.opengis.net/gml/3.2}metaDataProperty',
  {'about': 'contact',
   '{http://www.w3.org/1999/xlink}href': 'http://waterservices.usgs.gov'},
  None),
 ('{http://www.opengis.net/waterml/2.0}generationDate',
  {},
  '2019-06-03T0

In [27]:
[(elem.tag,elem.attrib,elem.text) for elem in try1.root.iter('{http://www.opengis.net/waterml/2.0}Collection')]

[('{http://www.opengis.net/waterml/2.0}Collection',
  {'{http://www.opengis.net/gml/3.2}id': 'C.USGS.02314495',
   '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.opengis.net/waterml/2.0 http://schemas.opengis.net/waterml/2.0/waterml2.xsd'},
  None)]

In [74]:
namespace={'ns0':"http://www.opengis.net/waterml/2.0",
           'ns1':"http://www.opengis.net/gml/3.2",
           'ns3':"http://www.w3.org/1999/xlink",
           'ns4':"http://www.opengis.net/om/2.0"}
for elem in try1.root.findall('ns0:observationMember',namespace):
    for elem2 in elem.findall('ns4:OM_Observation',namespace):
        print(elem2.tag)
        for elem3 in elem2.findall('ns4:result',namespace):
            for elem4 in elem3.findall('ns0:MeasurementTimeseries',namespace):
                for elem5 in elem4.findall('ns0:point',namespace):
                    for elem6 in elem5.findall('ns0:MeasurementTVP',namespace):
                        for elem7_time in elem6.findall('ns0:time',namespace):
                            print(elem7_time.text)
        

#[(elem.text,elem.attrib,elem.tag) for elem in try1.root.findall('ns0:observationMember',namespace) for elem2 in elem.]


{http://www.opengis.net/om/2.0}OM_Observation
2019-05-18T00:00:00-04:00
2019-05-18T00:15:00-04:00
2019-05-18T00:30:00-04:00
2019-05-18T00:45:00-04:00
2019-05-18T01:00:00-04:00
2019-05-18T01:15:00-04:00
2019-05-18T01:30:00-04:00
2019-05-18T01:45:00-04:00
2019-05-18T02:00:00-04:00
2019-05-18T02:15:00-04:00
2019-05-18T02:30:00-04:00
2019-05-18T02:45:00-04:00
2019-05-18T03:00:00-04:00
2019-05-18T03:15:00-04:00
2019-05-18T03:30:00-04:00
2019-05-18T03:45:00-04:00
2019-05-18T04:00:00-04:00
2019-05-18T04:15:00-04:00
2019-05-18T04:30:00-04:00
2019-05-18T04:45:00-04:00
2019-05-18T05:00:00-04:00
2019-05-18T05:15:00-04:00
2019-05-18T05:30:00-04:00
2019-05-18T05:45:00-04:00
2019-05-18T06:00:00-04:00
2019-05-18T06:15:00-04:00
2019-05-18T06:30:00-04:00
2019-05-18T06:45:00-04:00
2019-05-18T07:00:00-04:00
2019-05-18T07:15:00-04:00
2019-05-18T07:30:00-04:00
2019-05-18T07:45:00-04:00
2019-05-18T08:00:00-04:00
2019-05-18T08:15:00-04:00
2019-05-18T08:30:00-04:00
2019-05-18T08:45:00-04:00
2019-05-18T09:00:0

In [71]:
namespace={'ns0':"http://www.opengis.net/waterml/2.0",
           'ns1':"http://www.opengis.net/gml/3.2",
           'ns3':"http://www.w3.org/1999/xlink",
           'ns4':"http://www.opengis.net/om/2.0"}
for elem in try1.root.findall('ns0:observationMember',namespace):
    for elem2 in elem.findall('ns4:OM_Observation',namespace):
        #print(elem2.tag)
        for elem3 in elem2.findall('ns4:result',namespace):
            for elem4 in elem3.findall('ns0:MeasurementTimeseries',namespace):
                for elem5 in elem4.findall('ns0:point',namespace):
                    for elem6 in elem5.findall('ns0:MeasurementTVP',namespace):
                        times=elem6.findall('ns0:time',namespace)
                        for i in range(len(times)):
                            print(times[i].text)

#[(elem.text,elem.attrib,elem.tag) for elem in try1.root.findall('ns0:observationMember',namespace) for elem2 in elem.]


2019-05-18T00:00:00-04:00
2019-05-18T00:15:00-04:00
2019-05-18T00:30:00-04:00
2019-05-18T00:45:00-04:00
2019-05-18T01:00:00-04:00
2019-05-18T01:15:00-04:00
2019-05-18T01:30:00-04:00
2019-05-18T01:45:00-04:00
2019-05-18T02:00:00-04:00
2019-05-18T02:15:00-04:00
2019-05-18T02:30:00-04:00
2019-05-18T02:45:00-04:00
2019-05-18T03:00:00-04:00
2019-05-18T03:15:00-04:00
2019-05-18T03:30:00-04:00
2019-05-18T03:45:00-04:00
2019-05-18T04:00:00-04:00
2019-05-18T04:15:00-04:00
2019-05-18T04:30:00-04:00
2019-05-18T04:45:00-04:00
2019-05-18T05:00:00-04:00
2019-05-18T05:15:00-04:00
2019-05-18T05:30:00-04:00
2019-05-18T05:45:00-04:00
2019-05-18T06:00:00-04:00
2019-05-18T06:15:00-04:00
2019-05-18T06:30:00-04:00
2019-05-18T06:45:00-04:00
2019-05-18T07:00:00-04:00
2019-05-18T07:15:00-04:00
2019-05-18T07:30:00-04:00
2019-05-18T07:45:00-04:00
2019-05-18T08:00:00-04:00
2019-05-18T08:15:00-04:00
2019-05-18T08:30:00-04:00
2019-05-18T08:45:00-04:00
2019-05-18T09:00:00-04:00
2019-05-18T09:15:00-04:00
2019-05-18T0